In [1]:
import os
import fitz 
import pandas as pd
# import pytesseract
from PIL import Image
import re
from lang import transfer #文字轉換
from difflib import SequenceMatcher #字串模糊比對
from paddleocr import PaddleOCR,draw_ocr
ocr = PaddleOCR(use_angle_cls=True,use_gpu=False)


/home/rockey/.local/lib/python3.8/site-packages/paddle/base/framework.py:688: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(


[2024/04/10 15:42:02] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/home/rockey/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/rockey/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mo

In [80]:
class Source(object):
    def __ini__(self):
        pass
        # self.df=pd.read_excel('/mnt/c/Users/Administrator/Desktop/test_file/first_lot.xlsx')
        
    def table(self,df):
        # self.df=pd.read_excel('/mnt/c/Users/Administrator/Desktop/test_file/first_lot.xlsx')
        # self.df = df
        df_= df.loc[:,'報驗案號']
        df_ = df_.astype(str)
        return df_
    
    def pdfbase(self,df):
        Path_pdf = '/mnt/c/Users/Administrator/Desktop/test_file/化工科中標/All'#路徑
        pdf_list = os.listdir(Path_pdf) #每個檔案name
        for pdf in pdf_list: #資料夾內檔名跟excel 報驗案號比對
            PDF = pdf.replace('.pdf','')
            # print(PDF)
            df_=self.table(df)
            if PDF in df_.values:
                condition = df.loc[:,'報驗案號'].astype('str') == PDF  #step1 : 先批號核對
                # print(f'{PDF}檔案名稱與CSV檔相符')
            else:
                print(f'{PDF}案名稱與CSV檔不符')
        return condition

    def PDFtxt(self,PDFNAME):
        pdf_name = str(PDFNAME)
        # pdf_name = '7C403900011004'
        img_path = f'/home/rockey/pic/玩具案件/{pdf_name}.pdf'
        txt_result = ocr.ocr(img_path)
        # print(txt_result)
        return txt_result

    def pdf2txt(self,PDFNAME):
        txt_result = self.PDFtxt(PDFNAME)
        for i in range(len(txt_result[0])):
            txt = transfer(txt_result[0][i][1][0])
            return txt
            # print(transfer(txt_result[0][i][1][0]))

In [319]:
PDFNAME = 20403900003009
# df = pd.read_excel('/mnt/c/Users/Administrator/Desktop/test_file/first_lot.xlsx')
df = pd.read_excel('/mnt/c/Users/Administrator/Desktop/test_file/玩具案件/first_lot.xls')
source = Source()
# source.pdf2txt(PDFNAME)
condition = source.pdfbase(df) #excel對應案號
text =source.PDFtxt(PDFNAME) #辨識後未經處理內容
# text[0][0][1][0]

[2024/04/11 14:19:01] ppocr DEBUG: dt_boxes num : 18, elapsed : 0.32619333267211914
[2024/04/11 14:19:01] ppocr DEBUG: cls num  : 18, elapsed : 0.056700944900512695
[2024/04/11 14:19:03] ppocr DEBUG: rec_res num  : 18, elapsed : 2.120082139968872


0      False
1      False
2      False
3      False
4      False
       ...  
324    False
325    False
326    False
327    False
328    False
Name: 報驗案號, Length: 329, dtype: bool

In [438]:
try:
    [MARK,_name,position,Importer,Manuf,Address,LOT,warn_word,material_word,age_word] = [df.loc[condition,'字軌'].astype('str')
                ,df.loc[condition,'品名']
                ,df.loc[condition,'產地']
                ,df.loc[condition,'報驗義務人名稱']
                ,df.loc[condition,'委製商名稱']
                ,df.loc[condition,'報驗義務人地址']
                ,df.loc[condition,'批號']
                ,'警告及注意事項'
                ,['木頭','PP','PE','PVC','PET','PS','PC','PLA','合金','鋁合金','塑膠','橡膠']
                ,['2','3','4','6','7','三','八']
                ]
except Exception as e:
    print(f'{e}項目error')
warn_word


'批號'項目error


'警告及注意事項'

In [382]:
class identification(object):
    def __init__(self):
        self.text=text
        # self.result=result
    def result_message(self,pattern,text):
        result =  re.search(pattern,text)
        if result:
            return result.group(0)
        else:
            return None  # 或者返回适当的默认值
        
    def production_name(self,text):  #品名
        return self.result_message(r'品(.*?)名(.*?)$',text)
        
    def toy_name(self,text):  #玩具名稱
        return self.result_message(r'名(.*?)稱(.*?)$',text)
    
    def Position(self,text): #產地
        return self.result_message(r'產(.*?)地(.*?)$',text)
    
    def importer(self,text): #進口商名稱
        return self.result_message(r'進(.*?)口(.*?)商(.*?)$',text)
    
    def manufacturer(self,text): #委製商/製造商
        return self.result_message(r'製(.*?)商(.*?)$',text)
    
    def phone(self,text): #報驗義務電話
        return self.result_message(r'電(.*?)話(.*?)$',text)

    def address(self,text): #報驗義務地址  暫時無法   
        return self.result_message(r'地(.*?)址(.*?)$', text)
    
    def lot(self,text): #批號 
        return self.result_message(r'批(.*?)號(.*?)$',text)
    
    def M_mark(self,text): #Mark   
        return self.result_message(r'M(.*?)$',text)
    
    def warn_1(self,text): #警 告 及 注 意 事 項 
        return self.result_message(r'警(.*?)告(.*?)$',text)
    
    def warn_2(self,text): #警 告 及 注 意 事 項
        return self.result_message(r'注(.*?)意(.*?)$',text)
    
    def material(self,text): #成分
        return self.result_message(r'成(.*?)分(.*?)$',text)
    
    def age(self,text): #適用
        return self.result_message(r'適(.*?)年(.*?)$',text)
    
    def dis_age(self,text): #不適用
        return self.result_message(r'不(.*?)適(.*?)$',text)


In [384]:
#pdf結果逐行逐項
i = 0
# TXT = text[0]
NUMBER= text[0]  
Identification = identification()
for i in range(len(NUMBER)):
    print(f'i:{i}')
    TXT = text[0][i][1][0]
    # print(Identification.M_mark(TXT))
    # print(Identification.production_name(TXT))
    # # print(Identification.toy_name(TXT[i][1][0]))
    # print(Identification.Position(TXT))
    [mark_pdf,name_pdf,toy_name_pdf,local_pdf,Importer_pdf,Manuf_pdf
                ,PhoneNO_pdf,address_pdf,lot_pdf,warn_pdf_1
                ,warn_pdf_2,material_pdf,age_pdf,disage_pdf]=[Identification.M_mark(TXT)
                ,Identification.production_name(TXT)
                ,Identification.toy_name(TXT)
                ,Identification.Position(TXT)
                ,Identification.importer(TXT)
                ,Identification.manufacturer(TXT)
                ,Identification.phone(TXT)
                ,Identification.address(TXT)
                ,Identification.lot(TXT)
                ,Identification.warn_1(TXT)
                ,Identification.warn_2(TXT)
                ,Identification.material(TXT)
                ,Identification.age(TXT)
                ,Identification.dis_age(TXT)]
print(disage_pdf)

i:0
i:1
i:2
i:3
i:4
i:5
i:6
i:7
i:8
i:9
i:10
i:11
i:12
i:13
i:14
i:15
i:16
i:17
None


In [392]:
i = 12
TXT = text[0][i][1][0]
[mark_pdf,name_pdf,toy_name_pdf,local_pdf,Importer_pdf,Manuf_pdf
            ,PhoneNO_pdf,address_pdf,lot_pdf,warn_pdf_1
            ,warn_pdf_2,material_pdf,age_pdf,disage_pdf]=[Identification.M_mark(TXT)
            ,Identification.production_name(TXT)
            ,Identification.toy_name(TXT)
            ,Identification.Position(TXT)
            ,Identification.importer(TXT)
            ,Identification.manufacturer(TXT)
            ,Identification.phone(TXT)
            ,Identification.address(TXT)
            ,Identification.lot(TXT)
            ,Identification.warn_1(TXT)
            ,Identification.warn_2(TXT)
            ,Identification.material(TXT)
            ,Identification.age(TXT)
            ,Identification.dis_age(TXT)]

print(warn_pdf_1)
    # print(Identification.address(TXT[i][1][0]))
    # print(Identification.warn_1(TXT[i][1][0]))
    # print(Identification.warn_2(TXT[i][1][0]))
    # print(Identification.material(TXT[i][1][0]))
    # print(Identification.age(TXT[i][1][0]))
    # print(Identification.dis_age(TXT[i][1][0]))
    # print(Identification.lot(TXT[i][1][0]))

None


In [341]:
class Similar(object):
    def __init__(self):
        pass
    def count_matching_chars(self,database,pdf):
        # 去除空格并将字符串转换为集合
        # database_set = set(''.join(database))
        database = str(database)
        # print(database)
        database_set = set(''.join(database))
        pdf_set = set(''.join(pdf))
        count = len(database_set & pdf_set)
        return count
    
    def is_similar_count(self,item,database,pdf, min_matching_chars): #只要對超過2,便report
        # 计算匹配的字符数
        if pdf:
            # print('pdf:',pdf)
            matching_chars = self.count_matching_chars(database,pdf)
            if matching_chars >= min_matching_chars:
                print(f'{database}符合') 
                _sum =1
                
            else :
                print(f'{database}不符')
                _sum =0
        else:
            print(f'{database} None')
            _sum =0
        return _sum

    def warn_similar_age(self,database,pdf, min_matching_chars): #只要對超過2,便report
        if pdf: #如pdf 內文存在
            matching_chars = self.count_matching_chars(database,pdf)
            if matching_chars >= min_matching_chars:
                # sum = 1
                print(f'{database}以上符合') 
                _sum =1
                # Match = database
            else :
                # print(f'{database}不符')
                _sum =0
        else:
            print(f'{database} None')
            _sum =0
        return _sum

    def warn_similar_disage(self,item,database,pdf, min_matching_chars): #只要對超過2,便report
        # 计算匹配的字符数
        if pdf: #如pdf 內文存在
            # print('pdf:',pdf)
            matching_chars = self.count_matching_chars(database,pdf)
        # if matching_chars:
            if matching_chars >= min_matching_chars:
                # sum = 1
                print(f'{item}{database}以下符合') 
                _sum =1
            else :
                # print(f'{item}{database}不符')
                _sum =0
        else:
            print(f'{item}{database} None')
            _sum =0
            
        return _sum


In [435]:
similar = Similar()

def apply_warn_similar_material(material_word): #為了套用map所創函數
    return similar.warn_similar_age(material_word, material_pdf, 2)

def apply_warn_similar_age(age_word): #為了套用map所創函數
    age = similar.warn_similar_age(age_word, age_pdf, 1)
    # Age_condition = '合格' if any([x>0 for x in age]) else '不合格' 
    return age
    # return similar.warn_similar_age(age_word, age_pdf, 2)

def apply_warn_similar_disage(age_word): #為了套用map所創函數
    item = '年齡'
    return similar.warn_similar_disage(item,age_word, disage_pdf, 1)




def test():
    similar = Similar()
    Toy_1=similar.is_similar_count('品名:',_name,name_pdf,2)  
    Toy_2=similar.is_similar_count('品名:',_name,toy_name_pdf,2)  
    Toy_judgement = [Toy_1,Toy_2] 
    Toy_Score = 1 if any([x>0 for x in Toy_judgement]) else  0 #累計分數
    # Identification = identification(text)
    # condition = df.loc[:,'報驗案號'].astype('str') == PDF  #step1 : 先批號核對
    Position_=similar.is_similar_count('產地:',position,local_pdf,2)  #查產地
    Importer_ = similar.is_similar_count('義務人名稱:',Importer,Importer_pdf,3)  #查進口商名稱(暫)
    Manuf_=similar.is_similar_count('委製商名稱:',Manuf,Manuf_pdf,4)   #查委製商名稱
    LOT_ = similar.is_similar_count('批號:',LOT,lot_pdf,5)   #查報驗義務人地址
    address_ = similar.is_similar_count('報驗義務人地址:',Address,address_pdf,6)  
    # Phone_ = '電話符合' if len(PhoneNO_pdf)>2 else '電話不符'
    Warn_word_1=similar.warn_similar_count(warn_word,warn_pdf_1,2) #警語
    Warn_word_2=similar.warn_similar_count(warn_word,warn_pdf_2,2) #注意事項
    Warn_judgement = [Warn_word_1,Warn_word_2] 
    Warn_Score = 1 if any([x>0 for x in Warn_judgement]) else  0 #累計分數
    print(f'警告及注意事項合格') if any([x>0 for x in Warn_judgement]) else print(f'警告及注意事項不合格')
# similar.warn_similar_count(material_word,material_pdf,2) #成分
    Material_=list(map(apply_warn_similar_material, material_word))  #成分
    Material_Score = 1 if any([x>0 for x in Material_]) else  0 #累計分數
    # Material_judgement =print(f'合格') if any([x>0 for x in Material_]) else print(f'{material_pdf}不合格')
    Age_1=list(map(apply_warn_similar_age, age_word))  #適用年齡
    Age_Score = 1 if any([x>0 for x in Age_1]) else  0 #累計分數
    # Age_judgement = '合格' if any([x>0 for x in Age_1]) else print(f'年齡不合格') #只print不合格的部分
    Age_2=list(map(apply_warn_similar_disage, age_word))  #不適年齡
    # Disage_judgement =print(f'{age_pdf}合格') if any([x>0 for x in Age_2]) else print(f'{disage_pdf}不合格')
    Disage_Score = 1 if any([x>0 for x in Age_2]) else  0 #累計分數
    # Disage_judgement ='合格' if any([x>0 for x in Age_2]) else print(f'{disage_pdf}不合格') #只print不合格的部分
    # return Toy_1,

# test()

In [436]:
def itemScore(text,_name,position,Importer,warn_word,Manuf,Address,LOT,material_word,age_word):
    i = 0
    NUMBER= text[0]  
    Identification = identification()
    score = 0
    for i in range(len(NUMBER)):
        # print(f'i:{i}')
        TXT = text[0][i][1][0]
        name_pdf = Identification.production_name(TXT)
        local_pdf = Identification.Position(TXT)
        Importer_pdf = Identification.importer(TXT)
        Manuf_pdf = Identification.manufacturer(TXT)
        address_pdf = Identification.address(TXT)
        lot_pdf = Identification.lot(TXT)
        # material_pdf = Identification.material(TXT)
        warn_pdf_1 = Identification.warn_1(TXT)
        warn_pdf_2 = Identification.warn_2(TXT)
        Material_=list(map(apply_warn_similar_material, material_word))  #成分
        Age_1=list(map(apply_warn_similar_age, age_word))  #適用年齡
        Age_2=list(map(apply_warn_similar_disage, age_word))  #不適年齡
        #加總成績score,符合則+1,不符合則+0
        score += similar.is_similar_count('品名:',_name,name_pdf,3)  
        score += similar.is_similar_count('產地:',position,local_pdf,3)  
        score += similar.is_similar_count('義務人名稱:',Importer,Importer_pdf,3)  
        score += similar.is_similar_count('委製商名稱:',Manuf,Manuf_pdf,3) 
        score += similar.is_similar_count('報驗義務人地址:',Address,address_pdf,8) 
        score += similar.is_similar_count('批號:',LOT,lot_pdf,5)   #查報驗義務人地址
        score += similar.is_similar_count('警告:',warn_word,warn_pdf_1,2) #警語
        score += similar.is_similar_count('注意:',warn_word,warn_pdf_2,2) #警語
        score +=  1 if any([x>0 for x in Material_]) else  0 #成分或材質累計分數
        score +=  1 if any([x>0 for x in Age_1]) else  0 #"年齡X歲以上"累計分數
        score +=  1 if any([x>0 for x in Age_2]) else  0 #"年齡X歲以下"累計分數
        print(score)
    return score
    # sum = similar.is_similar_count('產地:',position,local_pdf,2)  #查產地
    # sum +=sum
    # print(sum)

# itemScore(text,_name,name_pdf)
# TXT = text[0][i][1][0]
# local_pdf = Identification.Position(TXT)
# pp.pprint(your_data)  # your_data 是你要打印的數據結構
score =itemScore(text,_name,position,Importer,warn_word,Manuf,Address,LOT,material_word, age_word)


木頭 None
PP None
PE None
PVC None
PET None
PS None
PC None
PLA None
合金 None
鋁合金 None
塑膠 None
橡膠 None
2 None
3 None
4 None
6 None
7 None
三 None
八 None
年齡2 None
年齡3 None
年齡4 None
年齡6 None
年齡7 None
年齡三 None
年齡八 None
319    模型車
Name: 品名, dtype: object不符
產地：中國 None
319    國際富豪汽車股份有限公司
Name: 報驗義務人名稱, dtype: object None
319    VOLVO CAR CORPORATION
Name: 委製商名稱, dtype: object None
319    台北市信義區信義路5段7號38樓-1
Name: 報驗義務人地址, dtype: object None
319    2401002.0
Name: 批號, dtype: float64 None
警告及注意事項 None
警告及注意事項 None
0
木頭 None
PP None
PE None
PVC None
PET None
PS None
PC None
PLA None
合金 None
鋁合金 None
塑膠 None
橡膠 None
2 None
3 None
4 None
6 None
7 None
三 None
八 None
年齡2 None
年齡3 None
年齡4 None
年齡6 None
年齡7 None
年齡三 None
年齡八 None
319    模型車
Name: 品名, dtype: object None
產地：中國符合
319    國際富豪汽車股份有限公司
Name: 報驗義務人名稱, dtype: object None
319    VOLVO CAR CORPORATION
Name: 委製商名稱, dtype: object None
319    台北市信義區信義路5段7號38樓-1
Name: 報驗義務人地址, dtype: object None
319    2401002.0
Name: 批號, dtype: float64 None
警告及注意事項 

In [439]:
# condition = Source()
# print(condition)
# PDFNAME = 20403900003009
# txt_result = text(PDFNAME)
# pdf2txt(txt_result)
# text[0][12][1][0]
# similar.is_similar_count('警語',warn_word,warn_pdf_1,2) #警語
# Age_1=list(map(apply_warn_similar_age, age_word))  #適用年齡
# 1 if any([x>0 for x in Age_1]) else  0 #"年齡X歲以上"累計分數
# similar.warn_similar_age(age_word, age_pdf, 1)
Age_1=list(map(apply_warn_similar_age, age_word))  #適用年齡
Age_1

2 None
3 None
4 None
6 None
7 None
三 None
八 None


[0, 0, 0, 0, 0, 0, 0]

In [ ]:
# MARK = df.loc[condition,'字軌'].astype('str') 

In [ ]:
# Identification = identification(txt_result)
condition = df.loc[:,'報驗案號'].astype('str') == PDF  #step1 : 先批號核對

In [49]:
#逐項列印呈現
# from lang import transfer #文字轉換
# def main():
#     print(transfer(result[0][0][1][0]))
#     print(transfer(result[0][1][1][0]))
#     print(transfer(result[0][2][1][0]))
#     print(transfer((result[0][3][1][0])))
#     print(transfer(result[0][4][1][0]))
#     print(transfer((result[0][5][1][0])))
#     print(transfer(result[0][6][1][0]))
#     print(transfer((result[0][7][1][0])))
#     print(transfer(result[0][8][1][0]))
#     print(transfer((result[0][9][1][0])))
    # print(result[0][10][1][0])
    # print(result[0][11][1][0])
    # print(result[0][12][1][0])
    # print(result[0][13][1][0])
    # print(result[0][14][1][0])

In [432]:
#pdf結果逐行逐項
i = 0
# TXT = text[0]
NUMBER= text[0]  
Identification = identification()
for i in range(len(NUMBER)):
    print(f'i:{i}')
    TXT = text[0][i][1][0]
    # print(Identification.Position(TXT))
    # print(Identification.M_mark(TXT))
    # print(Identification.production_name(TXT))
    # print(Identification.toy_name(TXT[i][1][0]))
    # print(Identification.Position(TXT))
    # print(Identification.importer(TXT[i][1][0]))
    # print(Identification.manufacturer(TXT[i][1][0]))
    # print(Identification.phone(TXT[i][1][0]))
    # print(Identification.address(TXT[i][1][0]))
    # print(Identification.warn_1(TXT[i][1][0]))
    # print(Identification.warn_2(TXT[i][1][0]))
    # print(Identification.material(TXT[i][1][0]))
    print(Identification.age(TXT))
    # print(Identification.dis_age(TXT[i][1][0]))
    # print(Identification.lot(TXT[i][1][0]))
    

i:0
None
i:1
None
i:2
None
i:3
適用年龄：6崴以上
i:4
None
i:5
None
i:6
None
i:7
None
i:8
None
i:9
None
i:10
None
i:11
None
i:12
None
i:13
None
i:14
None
i:15
None
i:16
None
i:17
None
